In [9]:
import pandas as pd
df_1 = pd.read_excel('ApacheSample.xlsx')
df_2 = pd.read_excel('junitSample.xlsx')
df_3 = pd.read_excel('okhttpSample.xlsx')
df_4 = pd.read_excel('retrofitSample.xlsx')
df_5 = pd.read_excel('springBootSample.xlsx')

In [50]:
df_1.head(15)

,id,repo_id,label,url,if_mulit_commit,message,new_message1,authorEmail,commitDate
0,3079,2,0,https://github.com/apache/dubbo/commit/1b2e6dc...,1,remove some magic value (#4752) <enter> <ente...,remove some magic value ( <pr_link> ) <enter>...,developer128,2019-08-06T07:23:45Z
1,4224,2,0,https://github.com/apache/dubbo/commit/9e9e778...,0,Optimize_hessian_desr_performance (#1705),Optimize_hessian_desr_performance ( <pr_link> ),developer130,2018-04-26T10:51:02Z
2,3817,2,0,https://github.com/apache/dubbo/commit/7f262f9...,0,simplify map empty judgment (#3376),simplify map empty judgment ( <pr_link> ),developer133,2019-01-29T05:38:51Z
3,2727,2,2,https://github.com/apache/dubbo/commit/ed7be11...,0,"constant names should be all uppercase, separa...","constant names should be all uppercase, separa...",developer134,2020-02-01T06:58:09Z
4,2613,2,0,https://github.com/apache/dubbo/commit/912303d...,0,collect async export services (#5905),collect async export services ( <pr_link> ),developer135,2020-04-01T14:49:58Z
5,2884,2,2,https://github.com/apache/dubbo/commit/dfa8499...,0,fix NetUtils.isPreferIPV6Address bug (#5238),fix <method_name> bug ( <pr_link> ),developer137,2019-11-08T05:20:57Z
6,3868,2,0,https://github.com/apache/dubbo/commit/c21cc05...,0,replace ServiceAnnotationBeanPostProcessor log...,replace <file_name> log.isInfoEnabled->log.i...,developer140,2018-10-31T03:29:54Z
7,2599,2,2,https://github.com/apache/dubbo/commit/317e62f...,0,Fix concurrency problems in RpcStatus and Roun...,Fix concurrency problems in <file_name> and ...,developer145,2020-04-07T02:43:21Z
8,4415,2,2,https://github.com/apache/dubbo/commit/6a77179...,0,Update OverrideServiceImpl.java <enter> <ente...,Update <file_name> <enter> fix the bug: cant ...,developer150,2017-10-10T09:06:02Z
9,4221,2,0,https://github.com/apache/dubbo/commit/71e04fb...,0,"Fix #1411 Java Locale use '_' split language, ...",Fix <issue_link> Java <iden> use '_' split lan...,developer152,2018-04-04T03:01:26Z


# Log model

In [99]:
import datetime
import logging
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def mylog():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logfile = "./log/" + str(datetime.datetime.now().month) + "-" + str(datetime.datetime.now().day) + "-" + str(
        datetime.datetime.now().hour) + "-" + str(datetime.datetime.now().minute) + \
              os.path.split(__file__)[-1].split(".")[0] + '.log'
    fileHandler = logging.FileHandler(logfile, mode='w', encoding='UTF-8')
    fileHandler.setLevel(logging.NOTSET)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fileHandler.setFormatter(formatter)
    logger.addHandler(fileHandler)
    return logger

# Define the preprocess_text function to perform text preprocessing steps
def preprocess_text(text):
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]

    # Perform stemming
    stemmer = PorterStemmer()
    stemmed_text = [stemmer.stem(word) for word in filtered_text]

    # Join the processed words back into a single string
    preprocessed_text = ' '.join(stemmed_text)
    
    return preprocessed_text



# Preprocess the commit message text
df_1['new_message1'] = df_1['new_message1'].apply(preprocess_text)

# Split the data into training and test sets
train_data, test_data = train_test_split(df_1, test_size=0.15, random_state=42)

# Extract features using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['new_message1'])
X_test = vectorizer.transform(test_data['new_message1'])

# Specify the column name for the labels
label_column = 'label'

# Get the corresponding labels for the training and test sets
y_train = train_data[label_column]
y_test = test_data[label_column]

# Train the model (replace with your desired classifier)
model = LinearSVC()
model.fit(X_train, y_train)

# Generate predictions for the test set
predictions = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)



Accuracy: 0.64


In [100]:
import numpy as np
predictions = model.predict(X_test)


predictions = np.array(predictions)




In [101]:
predictions

array([3, 0, 1, 0, 0, 0, 0, 2, 3, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 0, 0, 0, 0, 2, 0,
       0, 2, 3, 0, 0, 0])

In [102]:

import logging
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the function to configure logging
def mylog():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logfile = "./log/" + str(datetime.datetime.now().month) + "-" + str(datetime.datetime.now().day) + "-" + str(
        datetime.datetime.now().hour) + "-" + str(datetime.datetime.now().minute) + \
              os.path.split(__file__)[-1].split(".")[0] + '.log'
    fileHandler = logging.FileHandler(logfile, mode='w', encoding='UTF-8')
    fileHandler.setLevel(logging.NOTSET)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fileHandler.setFormatter(formatter)
    logger.addHandler(fileHandler)
    return logger


df = pd.read_excel('ApacheSample.xlsx')

# Split the data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.15, random_state=42)


label_column = 'label'  # Column name for the labels
message_column = 'new_message1'  # Column name for the preprocessed messages

# Get the corresponding labels for the test set
test_labels = test_data[label_column]

# Replace 'predictions' with your actual predictions for the test set
predictions = np.array([3, 0, 1, 0, 0, 0, 0, 2, 3, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 0])  # Replace 'model' with your trained model and 'X_test' with your test data

# Calculate accuracy for the whole test set
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy (Overall):", accuracy)

# Filter the test labels and predictions for c-why and c-what categories (label = 0)
c_why_what_labels = test_labels[test_labels == 0]
c_why_what_predictions = predictions[test_labels == 0]

# Calculate accuracy for c-why and c-what combined category
c_why_what_accuracy = accuracy_score(c_why_what_labels, c_why_what_predictions)
print("Accuracy (c-why and c-what):", c_why_what_accuracy)

# Verify the shapes of the training and test sets
print("Training set shape:", train_data.shape)
print("Test set shape:", test_data.shape)


Accuracy (Overall): 0.64
Accuracy (c-why and c-what): 0.8064516129032258
Training set shape: (278, 9)
Test set shape: (50, 9)


# for df_2

In [106]:
import datetime
import logging
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def mylog():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logfile = "./log/" + str(datetime.datetime.now().month) + "-" + str(datetime.datetime.now().day) + "-" + str(
        datetime.datetime.now().hour) + "-" + str(datetime.datetime.now().minute) + \
              os.path.split(__file__)[-1].split(".")[0] + '.log'
    fileHandler = logging.FileHandler(logfile, mode='w', encoding='UTF-8')
    fileHandler.setLevel(logging.NOTSET)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fileHandler.setFormatter(formatter)
    logger.addHandler(fileHandler)
    return logger

# Define the preprocess_text function to perform text preprocessing steps
def preprocess_text(text):
    if isinstance(text, str):
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
        stemmer = PorterStemmer()
        stemmed_text = [stemmer.stem(word) for word in filtered_text]
        preprocessed_text = ' '.join(stemmed_text)
        return preprocessed_text
    else:
        return ""

# Preprocess the commit message text
df_2['new_message1'] = df_2['new_message1'].apply(preprocess_text)

# Split the data into training and test sets
train_data, test_data = train_test_split(df_2, test_size=0.15, random_state=42)

# Extract features using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['new_message1'])
X_test = vectorizer.transform(test_data['new_message1'])

label_column = 'label'


y_train = train_data[label_column]
y_test = test_data[label_column]

model = LinearSVC()
model.fit(X_train, y_train)


predictions_2 = model.predict(X_test)

# Calculate accuracy
accuracy_2 = accuracy_score(y_test, predictions_2)
print("Accuracy:", accuracy_2)


Accuracy: 0.3877551020408163


In [107]:
predictions_2

array([1, 0, 0, 3, 3, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 3, 0, 0,
       3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 0, 3])

In [108]:
import logging
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the function to configure logging
def mylog():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logfile = "./log/" + str(datetime.datetime.now().month) + "-" + str(datetime.datetime.now().day) + "-" + str(
        datetime.datetime.now().hour) + "-" + str(datetime.datetime.now().minute) + \
              os.path.split(__file__)[-1].split(".")[0] + '.log'
    fileHandler = logging.FileHandler(logfile, mode='w', encoding='UTF-8')
    fileHandler.setLevel(logging.NOTSET)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fileHandler.setFormatter(formatter)
    logger.addHandler(fileHandler)
    return logger




# Split the data into training and test sets
train_data, test_data = train_test_split(df_2, test_size=0.15, random_state=42)


label_column = 'label'  # Column name for the labels
message_column = 'new_message1'  # Column name for the preprocessed messages

# Get the corresponding labels for the test set
test_labels = test_data[label_column]

# Replace 'predictions' with your actual predictions for the test set
predictions =  np.array([1, 0, 0, 3, 3, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 3, 0, 0,
       3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 0, 3])  # Replace 'model' with your trained model and 'X_test' with your test data

# Calculate accuracy for the whole test set
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy (Overall):", accuracy)

# Filter the test labels and predictions for c-why and c-what categories (label = 0)
c_why_what_labels = test_labels[test_labels == 0]
c_why_what_predictions = predictions[test_labels == 0]

# Calculate accuracy for c-why and c-what combined category
c_why_what_accuracy = accuracy_score(c_why_what_labels, c_why_what_predictions)
print("Accuracy (c-why and c-what):", c_why_what_accuracy)

# Verify the shapes of the training and test sets
print("Training set shape:", train_data.shape)
print("Test set shape:", test_data.shape)


Accuracy (Overall): 0.3877551020408163
Accuracy (c-why and c-what): 0.6666666666666666
Training set shape: (274, 9)
Test set shape: (49, 9)
